In [1]:
import numpy as np
import scipy.io as sio 
import scipy.signal as sig
from matplotlib import pyplot as plt
%matplotlib inline
import random 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, Normalizer
import tensorflow 
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Reshape, Conv1D
from tensorflow.keras.regularizers import l2
import seaborn as sns

In [2]:
dat = [sio.loadmat(f'Data/BCICIV_4_mat/sub{x}_comp.mat',struct_as_record=True) for x in range(1,4)]
tdat = [sio.loadmat(f'Data/sub{x}_testlabels.mat',struct_as_record=True) for x in range(1,4)]

train_data = [dat[x]['train_data'] for x in range(3)]
test_data = ([dat[x]['test_data'] for x in range(3) ])
train_dg = ([dat[x]['train_dg'] for x in range(3) ])
test_dg = ([tdat[x]['test_dg'] for x in range(3) ])

train_samples = [train_data[i].shape[0] for i in range(3)]
channels = [train_data[i].shape[1] for i in range(3)]
test_samples = [test_data[i].shape[0] for i in range(3)]
channel_train_data = [np.transpose(train_data[i],(1,0)) for i in range(3)]
channel_test_data = [np.transpose(test_data[i],(1,0)) for i in range(3)]
finger_train_data = [np.transpose(train_dg[i],(1,0)) for i in range(3)]
finger_test_data = [np.transpose(test_dg[i],(1,0)) for i in range(3)]

sampling_frequency = 1000

In [3]:
# Merge
merch_train = []
merch_test = []
for i in range(max(channels)):
    merch = []
    for c in range(len(channel_train_data)):
        if(i >= channels[c]):
            for j in range(train_samples[c]):
                merch.append(0)
        else:
            for val in channel_train_data[c][i]:
                merch.append(val)
    merch_train.append(merch)

for i in range(max(channels)):
    merch = []
    for c in range(len(channel_test_data)):
        if(i >= channels[c]):
            for j in range(test_samples[c]):
                merch.append(0)
        else:
            for val in channel_test_data[c][i]:
                merch.append(val)
    merch_test.append(merch)

In [34]:
merf_train = []
for i in range(5):
    merf = []
    for f in finger_train_data:
        for val in f[i]:
            merf.append(val)
    merf_train.append(merf)

merf_test = []
for i in range(5):
    merf = []
    for f in finger_test_data:
        for val in f[i]:
            merf.append(val)
    merf_test.append(merf)

In [35]:
print(len(merch_train))
print(len(merch_train[0]))
print(len(merch_test))
print(len(merf_train))
print(len(merf_train[0]))
print(len(merf_test))

64
1200000
64
5
1200000
5


In [12]:
# def assign_states(finger_data):
#     # State assignment : (0 : rest, 1-5 finger flexed)
#     dsamples = len(finger_data[0])
#     states = [None]*dsamples
#     threshold_1,threshold_2  = 2.0,1.0
#     for i in range(dsamples):
#         flex,rest = 0,0
#         for j in range(5):
#             if finger_data[j][i] >= threshold_1:
#                 states[i] = j + 1
#                 flex += 1
#             elif finger_data[j][i] < threshold_2:
#                 rest += 1
#         if states[i] == None:
#             if rest:
#                 states[i] = 0

#     return states

In [13]:
# merf_train_states = assign_states(merf_train)
# merf_test_states = assign_states(merf_test)

In [14]:
# print(len(merf_train_states))

1200000


In [36]:
trc = np.array(merch_train)
trf = np.array(merf_train)

tec = np.array(merch_test)
tef = np.array(merf_test)

In [16]:
# Finger One-Hot Encoding

# trf = trf.reshape(len(trf), 1)
# trf = OneHotEncoder(sparse=False).fit_transform(trf)

# tef = tef.reshape(len(tef), 1)
# tef = OneHotEncoder(sparse=False).fit_transform(tef)

In [37]:
trc = trc.T
tec = tec.T
print(trc.shape)
print(tec.shape)

(1200000, 64)
(600000, 64)


In [39]:
trf = trf.T
tef = tef.T
trf = trf.reshape(len(trf),5)
tef = tef.reshape(len(tef),5)

In [40]:
# ECoG + Finger Normalization

trc = Normalizer().fit(trc).transform(trc)
tec = Normalizer().fit(tec).transform(tec)
trf = Normalizer().fit(trf).transform(trf)
tef = Normalizer().fit(tef).transform(tef)

In [41]:
# Reshape to 3D Structure for Conv and BiLSTM

trc = trc.reshape(len(trc),1,len(trc[0]))
tec = tec.reshape(len(tec),1,len(tec[0]))

In [20]:
np.random.seed(0)

In [21]:
# trc_finger_nonhotencoded = np.argmax(trf,axis=1)
# tec_finger_nonhotencoded = np.argmax(tef,axis=1)

In [43]:
# MODEL

model = keras.Sequential()
model.add(Conv1D(64, 1, activation='relu', input_shape=(1,64)))
model.add(Conv1D(48, 1, activation='relu', ))
forward_layer = LSTM(12, activation='relu')
backward_layer = LSTM(12, activation='relu', go_backwards=True)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
model.add(Dense(5, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['mae','mse'])
model.summary()

history = model.fit(trc, trf, batch_size = 2000, epochs=5, validation_split=0.05)

predictions_tr = model.predict(trc)
predictions_te = model.predict(tec)
# predictions_tr = np.argmax(predictions_tr, axis = 1)
# predictions_te = np.argmax(predictions_te, axis = 1)
np.savetxt('predictions_tr_combined.txt',predictions_tr)
np.savetxt('predictions_te_combined.txt',predictions_te)


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 1, 64)             4160      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1, 48)             3120      
_________________________________________________________________
bidirectional_6 (Bidirection (None, 24)                5856      
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 125       
Total params: 13,261
Trainable params: 13,261
Non-trainable params: 0
_________________________________________________________________
Train on 1140000 samples, validate on 60000 samples
Epoch 1/5
1140000/1140000 [==============================] - 38s 33us/sample - loss: -90146.7524 - mae: 0.5158 - mse: 0.4589 - val_loss: 373185.7218 - val_mae: 0.4993 - val_mse: 0.4571
Epoch 2/5
1

In [2]:
corr = np.corrcoef(tef, predictions_te)

NameError: name 'np' is not defined